# LangChain 实战：房产销售聊天机器人

## 使用 GPT-4 构造销售话术数据

ChatGPT 分享链接：https://chat.openai.com/share/f3e4b9b0-95fb-4c6a-a3c7-f901dd194c91


使用 ChatGPT 构造销售数据的 Prompt 示例：

```
你是中国顶级的房地产销售，现在培训职场新人，请给出100条实用的销售话术。

每条销售话术以如下格式给出：
[客户问题]
[销售回答]

```

GPT-4 回复结果：

```
在房地产销售行业中，精心准备的销售话术可以极大地提高成交率。以下是一些适用于中国房地产销售的实用话术：

1.  
[客户问题] 这个小区交通便利吗？
[销售回答] 当然了，这个小区距离地铁站只有几分钟的步行距离，而且附近有多条公交线路，非常方便。

2.  
[客户问题] 我担心楼下太吵。
[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。

3.  
[客户问题] 我看房价还在涨，这个投资回报怎么样？
[销售回答] 这个区域未来有大量的商业和基础设施建设，所以从长期来看，投资回报非常有保证。

4.  
[客户问题] 有没有学校？
[销售回答] 附近有多所优质的学校，非常适合有孩子的家庭。

5.  
[客户问题] 物业管理怎么样？
[销售回答] 我们的物业管理得到了业主一致好评，服务非常到位。

6.  
[客户问题] 我想要南向的房子。
[销售回答] 很好，我们确实有一些朝南的单位，它们的采光特别好。

7.  
[客户问题] 这个小区安全吗？
[销售回答] 当然，我们24小时安保巡逻，还有先进的监控系统。

8.  
[客户问题] 预计什么时候交房？
[销售回答] 根据目前的进度，我们预计将在明年底交房。

9.  
[客户问题] 我不想要一楼的房子。
[销售回答] 我理解您的顾虑，我们还有多个楼层的房源可以选择。

10.  
[客户问题] 有优惠吗？
[销售回答] 当然，如果您现在下订，我们可以给您一些优惠。
```


## 使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 [real_estate_sales_data.txt](real_estate_sales_data.txt) 文件中

In [5]:
with open("real_car_sales_data.txt",'r', encoding='utf-8') as f:
    real_estate_sales = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [6]:
from langchain.text_splitter import CharacterTextSplitter

In [7]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [8]:
docs = text_splitter.create_documents([real_estate_sales])

Created a chunk of size 102, which is longer than the specified 100
Created a chunk of size 133, which is longer than the specified 100
Created a chunk of size 115, which is longer than the specified 100


In [9]:
docs[0]

Document(page_content='[客户问题]：这款车的油耗怎么样？\n[销售回答]：这款车采用了先进的经济燃烧技术，不仅保证了动力的输出，同时也得到了极佳的油耗表现，在同级别车型中属于非常优秀的水平。')

In [10]:
len(docs)

19

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [11]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings())

C:\Scoop\apps\miniconda3\current\envs\openai-quickstart\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [12]:
query = "做贷款需要什么条件？"

In [13]:
answer_list = db.similarity_search(query)

In [14]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题]：贷款购车的手续复杂吗？
[销售回答]：非常简单，我们会有专人帮助您处理所有贷款手续，并且根据您的财务状况为您量身定制最合适的还款计划，让您轻松拥车。

[客户问题]：我是第一次买车，有什么好的建议吗？
[销售回答]：非常荣幸为您提供服务，首先我们会根据您的实际需求和预算来推荐最适合您的车型。其次，您可以放心我们的试驾体验，让您更好地了解车辆性能和舒适度。最后，我们会提供全程的售后服务，确保您购车后的使用体验。

[客户问题]：这车的再售值怎么样？
[销售回答]：我们品牌的车型保值率一直在行业内处于领先地位，由于其可靠性和耐用性，市场需求一直很旺盛，您可以放心。

[客户问题]：我有特定的颜色需求，定制需要很久吗？
[销售回答]：我们有多种标准颜色可以选择，如果您需要特别定制，我们将尽快安排生产，一般情况下交车时间都会在您的预期之内。



In [15]:
db.save_local("real_car_sale")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [22]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [23]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001571F1BA440>, search_kwargs={'k': 3})

In [24]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

C:\Scoop\apps\miniconda3\current\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[客户问题]：贷款购车的手续复杂吗？
[销售回答]：非常简单，我们会有专人帮助您处理所有贷款手续，并且根据您的财务状况为您量身定制最合适的还款计划，让您轻松拥车。

[客户问题]：我是第一次买车，有什么好的建议吗？
[销售回答]：非常荣幸为您提供服务，首先我们会根据您的实际需求和预算来推荐最适合您的车型。其次，您可以放心我们的试驾体验，让您更好地了解车辆性能和舒适度。最后，我们会提供全程的售后服务，确保您购车后的使用体验。

[客户问题]：这车的再售值怎么样？
[销售回答]：我们品牌的车型保值率一直在行业内处于领先地位，由于其可靠性和耐用性，市场需求一直很旺盛，您可以放心。



In [25]:
docs = topK_retriever.get_relevant_documents("车的保值咋样？")

In [26]:
for doc in docs:
    print(doc.page_content + "\n")

[客户问题]：这车的再售值怎么样？
[销售回答]：我们品牌的车型保值率一直在行业内处于领先地位，由于其可靠性和耐用性，市场需求一直很旺盛，您可以放心。

[客户问题]：我担心新车的维护成本。
[销售回答]：我们提供的是全面的售后服务计划，确保您购车后的维护既便捷又经济，配件价格透明且具有高性价比，您可以安心。

[客户问题]：我平时在市区开车，这款车好停吗？
[销售回答]：当然，这款车的车身尺寸适中，搭载了泊车辅助系统，包括倒车摄像头和前后停车传感器，即使在繁忙的市区也能轻松驾驭。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [27]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [29]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

C:\Scoop\apps\miniconda3\current\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


### 提取向量数据库中的`销售回答`

In [20]:
docs = retriever.get_relevant_documents(query)

In [21]:
docs[0].page_content

'[客户问题] 我担心楼下太吵。\n[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。'

In [22]:
docs[0].page_content.split("[销售回答] ")

['[客户问题] 我担心楼下太吵。\n', '这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。']

In [23]:
ans = docs[0].page_content.split("[销售回答] ")[-1]

In [24]:
ans

'这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。'

#### 尝试各种问题

In [25]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答] ")[-1] for doc in docs]

    return ans_list

In [26]:
query = "我想离医院近点"

print(sales(query))

[]


/root/miniconda3/envs/langchain/lib/python3.10/site-packages/langchain/schema/vectorstore.py:287: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


In [27]:
print(sales(query, 0.75))

['有的，距离我们小区不远就有几家大型综合医院。', '是的，附近有多家大型医院，医疗资源非常丰富。']


In [28]:
query = "价格200万以内"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

score:0.8 ans: []

score:0.75 ans: []



/root/miniconda3/envs/langchain/lib/python3.10/site-packages/langchain/schema/vectorstore.py:287: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.75
  warnings.warn(


score:0.5 ans: ['我们有不同户型和付款方案，一定有适合您预算的。', '我们的房子位于黄金地段，升值潜力很大，转卖起来不会有问题。', '定金金额取决于您选择的房型和付款方式，我们可以详细为您解释。', '这个区域正在快速发展，未来的升值潜力非常大。']



#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [29]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [31]:
qa_chain({"query": "你们小区有200万的房子吗？"})

{'query': '你们小区有200万的房子吗？',
 'result': '对不起，我无法回答这个问题，因为我是一个人工智能，没有实时的房地产信息。建议你直接联系房地产经纪人或者查阅相关房地产网站获取信息。'}

In [32]:
qa_chain({"query": "小区吵不吵"})

{'query': '小区吵不吵',
 'result': '这个小区特别注重居住体验，有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。所以，小区应该不会太吵。'}

In [33]:
print(sales("小区吵不吵"))

['这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。']


## 加载 FAISS 向量数据库已有结果

In [34]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("real_estates_sale", OpenAIEmbeddings())

In [35]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [36]:
qa_chain({"query": "我想买别墅，你们有么"})

{'query': '我想买别墅，你们有么',
 'result': '对不起，我无法提供这样的服务。我是一个人工智能，我可以帮助回答问题，但我不能出售商品或房产。'}

In [37]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [38]:
qa_chain({"query": "我想买别墅，你们有么"})



> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '我想买别墅，你们有么', 'result': '对不起，我不能帮助你购买别墅，因为我是一个人工智能，没有提供房地产服务的功能。'}

In [39]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [40]:
result = qa_chain({"query": "我想买别墅，你们有么"})



> Entering new StuffDocumentsChain chain...

> Finished chain.


In [41]:
result

{'query': '我想买别墅，你们有么',
 'result': '对不起，我不能帮你购买别墅。我是一个AI助手，我主要用来提供信息和回答问题。',
 'source_documents': []}